In [ ]:
SQL запити
STUDENT_ID	DEPARTMENT_ID	TEACHER_ID	START_DATE	FULL_NAME	SCHOLARSHIP
1	50	100	8-1-2019	Біляєв Матвій Артемович	700
2	50	100	8-22-2019	Блажевич Ігор Юрійович	690
3	50	100	8-1-2019	Валієва Руфіна Рафаелівна	690
4	60	200	8-22-2019	Височин Олександр Андрійович	730
5	60	200	8-1-2019	Гріненко Олексій Олексійович	740
6	70	300	8-22-2019	Жигляєв Родіон Олексійович	790
7	60	200	8-1-2019	Журавльова Анастасія Сергіївна	730
8	60	200	8-29-2019	Зіборов Кирило Вікторович	720
9	70	300	7-29-2019	Колосов Дмитро Григорович	800
10	70	300	7-28-2019	Червоних Олексій Володимирович	790

DEPARTMENT_ID	DEPARTMENT_NAME	BLOCK_ID
50	Менеджмент	1
60	Соціологія	1
70	Історія	2
80	Математика	2
90	Філологія	3


1)	Створити таблицю students з полями:
STUDENT_ID - ІД студента
DEPARTMENT_ID - ІД кафедри
TEACHER_ID -ІД викладача
START_DATE -Дата надходження
FULL_NAME -ПІБ студента
SCHOLARSHIP - Стипендія студента

create table natali_1g8.students
(STUDENT_ID number,
DEPARTMENT_ID number,
TEACHER_ID number,
START_DATE date,
FULL_NAME varchar2(100),
SCHOLARSHIP number)
;
comment on column natali_1g8.students.student_id is 'ІД студента';
comment on column natali_1g8.students.department_id is 'ІД кафедри';
comment on column natali_1g8.students.teacher_id is 'ІД викладача';
comment on column natali_1g8.students.start_date is 'Дата надходження';
comment on column natali_1g8.students.full_name is 'ПІБ студента ';
comment on column natali_1g8.students.scholarship is 'Стипендія студента';


2)	Створити таблицю departments з полями
DEPARTMENT_ID – ІД кафедри
DEPARTMENT_NAME-Назва кафедри
BLOCK_ID – ІД корпусу

create table natali_1g8.departments
(DEPARTMENT_ID number,
DEPARTMENT_NAME varchar2(100),
BLOCK_ID number)
;
comment on column natali_1g8.departments.department_id is ' ІД кафедри ';
comment on column natali_1g8.departments.department_name is ' Назва кафедри ';
comment on column natali_1g8.departments.block_id is ' ІД корпусу ';

3)Вивести всі поля з таблиці студентів, але замість стовпця DEPARTMENT_ID потрібно вивести поле DEPARTMENT_NAME з таблиці департаментів.

select ss.student_id,
dp.department_name,
ss.teacher_id,
ss.start_date,
ss.full_name,
ss.scholarship
from natali_1g8.students ss
left outer join natali_1g8.departments dp
on ss.department_id=dp.department_id;

4)виведи список назв кафедр, де кількість студентів більша, або дорівнює 4

select dp.department_name,
count(ss.student_id) as count_students
from natali_1g8.students ss
left outer join natali_1g8.departments dp
on ss.department_id=dp.department_id
group by dp.department_name
having count(ss.student_id) > = 4;

5) вивести список студентів, у яких максимальна стипендія, в рамках своєї кафедри

select
ss.full_name as student_name,
dp.department_name,
mx.max_scholarship
from natali_1g8.departments dp

join ( select max(ss.scholarship) as max_scholarship,
ss.department_id
from natali_1g8.students ss
group by ss.department_id)  mx

on dp.department_id=mx.department_id
join natali_1g8.students ss
on ss.scholarship =mx.max_scholarship;

або так:

select s.student_id, s.full_name, s.department_id, s.scholarship
from natali_1g8.students s
where s.scholarship in (select max(st.scholarship) --,st.department_id
from natali_1g8.students st
group by st.department_id);




6) вивести назви департаментів по яким є студенти

select  dp.department_name,
count(ss.student_id) as cnt_students
from natali_1g8.departments dp
left outer join natali_1g8.students ss
on dp.department_id=ss.department_id
group by dp.department_name
having count(ss.student_id)>0;

або так:

select
d.department_name
from
departments d
where
exists (
select 1
from students s
where s.department_id = d.department_id );

7) вивести студентів, які вступили до університету  у липні 2019 року, додати стовпець з назвою місяця

select  ss.full_name,
ss.start_date,
to_char(ss.start_date, 'month') as month
from natali_1g8.students ss
where ss.start_date between to_date('01.07.2019', 'dd.mm.yyyy')and to_date('31.07.2019', 'dd.mm.yyyy');

8) вставити в таблицю дублікати по студентам, та написати запит, який виводить рядки з дублікатами

insert into natali_1g8.students
select 1, 50, 100, to_date('01.08.2019', 'dd.mm.yyyy'), 'Біляєв Матвій Артемович', 700
from dual
union
select 2, 50, 100, to_date('22.08.2019', 'dd.mm.yyyy'), 'Блажевич Ігор Юрійович', 690
from dual
union
select 3, 50, 100, to_date('01.08.2019', 'dd.mm.yyyy'), 'Валієва Руфіна Рафаелівна', 690
from dual;
commit;
select student_id, department_id, teacher_id, start_date, full_name, scholarship
from students
group by student_id, department_id, teacher_id, start_date, full_name, scholarship
having count(student_id) > 1;

9) видалити дублюючи записи студентів

delete from  natali_1g8.students ss
where rowid in (select min(rowid)
from students st
group by student_id, department_id, teacher_id, start_date, full_name, scholarship
having count(*)>1);
